In [17]:
from data import Data
from base_types import DataType
import json
from utils import milliseconds_to_date

usd_name = 'BUSD'
token_name='LUNA2'
start_str = milliseconds_to_date(1654052700000) + ' UTC+8'
end_str = milliseconds_to_date(1654768499999 + 1) + ' UTC+8'
data = Data(token_name+usd_name, DataType.INTERVAL_1MINUTE, start_str=start_str, end_str=end_str, is_futures=True)
print(data.start_time())
print(data.end_time())
data = data.data
data.head()

1654052700000
1654768499999


,open_time,open,high,low,close,volume,close_time,quote_assert_volume,number_of_trades,taker_buy_volume,taker_buy_quote_asset_volume
0,1654052700000,8.4002,8.4092,8.3901,8.4050,6934,1654052759999,58247.3318,180,2559,21504.7585
1,1654052760000,8.4049,8.4220,8.3931,8.4060,2663,1654052819999,22393.4372,109,1947,16379.0442
2,1654052820000,8.4020,8.4250,8.4019,8.4022,1627,1654052879999,13680.4350,122,746,6275.8895
3,1654052880000,8.4060,8.4300,8.4018,8.4200,2159,1654052939999,18179.6477,101,1424,11996.8939
4,1654052940000,8.4102,8.4200,8.4100,8.4101,3480,1654052999999,29282.0332,187,1758,14799.9585


In [15]:
base_path = '.\\log\\{}_start_{}_end_{}_'.format(token_name+usd_name, data.start_time(), data.end_time())
trade_info_path = base_path + 'trade_info.json'
vertices_path = base_path + 'vertices.json'

with open(trade_info_path, 'r') as f:
    json_data = f.read()
    trade_info = json.loads(json_data)

with open(vertices_path, 'r') as f:
    json_data = f.read()
    vertices = json.loads(json_data)

print(trade_info.keys())
print(vertices.keys())

dict_keys(['buy_time', 'buy_price', 'sell_time', 'sell_price'])
dict_keys(['top_time', 'top_value', 'bottom_time', 'bottom_value'])


In [49]:
import numpy as np
import pandas as pd
# data['TR'] = data[['high', 'low']].apply(lambda x: x['high'] - x['low'], axis=1)
# data['MA4'] = data['close'].rolling(4).mean()
top_idx = -1
bottom_idx = -1
top_time = vertices['top_time']
top_value = vertices['top_value']
bottom_time = vertices['bottom_time']
bottom_value = vertices['bottom_value']

new_col = ['last_top', 'step_after_top', 'last_bottom', 'step_after_bottom','is_up', 'cycle_step']
data = pd.concat([data, 
    pd.DataFrame(columns=new_col)], sort=False)

def fun(x):
    global top_idx, bottom_idx
    open_time = x['open_time']

    if top_idx + 1 < len(top_time) and open_time >= int(top_time[top_idx + 1]) + 60000 * 3:
        top_idx += 1

    if bottom_idx + 1 < len(bottom_time) and open_time >= int(bottom_time[bottom_idx + 1]) + 60000 * 3:
        bottom_idx += 1
    
    if top_idx >= 0:
        last_top = top_value[top_idx]
        step_after_top = (open_time - top_time[top_idx]) / 60000
    else:
        last_top = np.nan
        step_after_top = np.nan
        
    if bottom_idx >= 0:
        last_bottom = bottom_value[bottom_idx]
        step_after_bottom = (open_time - bottom_time[bottom_idx]) / 60000
    else:
        last_bottom = np.nan
        step_after_bottom = np.nan
    
    is_up = 1 if step_after_top > step_after_bottom else 0
    cycle_step = step_after_top - step_after_bottom
    cycle_step = cycle_step if cycle_step >= 0 else -cycle_step

    return last_top, step_after_top, last_bottom, step_after_bottom, is_up, cycle_step

data[new_col] = data.apply(
    lambda x: fun(x), axis=1, result_type="expand")
    
data.head(10)

,open_time,open,high,low,close,volume,close_time,quote_assert_volume,number_of_trades,taker_buy_volume,taker_buy_quote_asset_volume,MA4,TR,last_top,step_after_top,last_bottom,step_after_bottom,is_up,cycle_step
0,1.654053e+12,8.4002,8.4092,8.3901,8.4050,6934.0,1.654053e+12,58247.3318,180.0,2559.0,21504.7585,NaN,0.0191,NaN,NaN,NaN,NaN,0.0,NaN
1,1.654053e+12,8.4049,8.4220,8.3931,8.4060,2663.0,1.654053e+12,22393.4372,109.0,1947.0,16379.0442,NaN,0.0289,NaN,NaN,NaN,NaN,0.0,NaN
2,1.654053e+12,8.4020,8.4250,8.4019,8.4022,1627.0,1.654053e+12,13680.4350,122.0,746.0,6275.8895,NaN,0.0231,NaN,NaN,NaN,NaN,0.0,NaN
3,1.654053e+12,8.4060,8.4300,8.4018,8.4200,2159.0,1.654053e+12,18179.6477,101.0,1424.0,11996.8939,8.408300,0.0282,NaN,NaN,8.3901,3.0,0.0,NaN
4,1.654053e+12,8.4102,8.4200,8.4100,8.4101,3480.0,1.654053e+12,29282.0332,187.0,1758.0,14799.9585,8.409575,0.0100,NaN,NaN,8.3901,4.0,0.0,NaN
5,1.654053e+12,8.4101,8.4200,8.4100,8.4100,3988.0,1.654053e+12,33561.4845,98.0,2170.0,18269.2241,8.410575,0.0100,NaN,NaN,8.3901,5.0,0.0,NaN
6,1.654053e+12,8.4174,8.4258,8.4101,8.4258,8276.0,1.654053e+12,69686.3413,139.0,8229.0,69290.9199,8.416475,0.0157,8.43,3.0,8.3901,6.0,0.0,3.0
7,1.654053e+12,8.4290,8.4800,8.4290,8.4600,3874.0,1.654053e+12,32758.0052,208.0,2488.0,21032.8639,8.426475,0.0510,8.43,4.0,8.4100,3.0,1.0,1.0
8,1.654053e+12,8.4703,8.4850,8.4438,8.4672,3082.0,1.654053e+12,26088.5500,165.0,1737.0,14707.0831,8.440750,0.0412,8.43,5.0,8.4100,4.0,1.0,1.0
9,1.654053e+12,8.4717,8.4734,8.4300,8.4468,4528.0,1.654053e+12,38224.6461,173.0,1090.0,9201.2583,8.449950,0.0434,8.43,6.0,8.4100,5.0,1.0,1.0


In [50]:
data.tail(30)

,open_time,open,high,low,close,volume,close_time,quote_assert_volume,number_of_trades,taker_buy_volume,taker_buy_quote_asset_volume,MA4,TR,last_top,step_after_top,last_bottom,step_after_bottom,is_up,cycle_step
11900,1.654767e+12,2.9508,2.9582,2.9438,2.9504,119227.0,1.654767e+12,3.517047e+05,1389.0,59295.0,1.749293e+05,2.947225,0.0144,2.9863,7.0,2.9568,10.0,0.0,3.0
11901,1.654767e+12,2.9504,2.9654,2.9494,2.9558,147851.0,1.654767e+12,4.373519e+05,1600.0,76154.0,2.253024e+05,2.948250,0.0160,2.9863,8.0,2.9210,3.0,1.0,5.0
11902,1.654767e+12,2.9553,2.9891,2.9513,2.9750,292434.0,1.654767e+12,8.686248e+05,2680.0,153656.0,4.566230e+05,2.957825,0.0378,2.9655,3.0,2.9210,4.0,0.0,1.0
11903,1.654767e+12,2.9750,2.9751,2.9601,2.9667,120717.0,1.654767e+12,3.582174e+05,1477.0,65037.0,1.929873e+05,2.961975,0.0150,2.9655,4.0,2.9210,5.0,0.0,1.0
11904,1.654767e+12,2.9668,2.9700,2.9513,2.9666,221422.0,1.654767e+12,6.559957e+05,1830.0,104634.0,3.100725e+05,2.966025,0.0187,2.9655,5.0,2.9494,3.0,1.0,2.0
11905,1.654767e+12,2.9666,3.0000,2.9595,3.0000,337522.0,1.654767e+12,1.006301e+06,3494.0,194003.0,5.785151e+05,2.977075,0.0405,2.9891,3.0,2.9494,4.0,0.0,1.0
11906,1.654767e+12,2.9992,3.0448,2.9976,3.0129,504324.0,1.654767e+12,1.524160e+06,4253.0,296730.0,8.967141e+05,2.986550,0.0472,2.9891,4.0,2.9494,5.0,0.0,1.0
11907,1.654767e+12,3.0138,3.0583,3.0102,3.0465,363931.0,1.654767e+12,1.105409e+06,2865.0,224918.0,6.832072e+05,3.006500,0.0481,2.9891,5.0,2.9513,3.0,1.0,2.0
11908,1.654767e+12,3.0477,3.0585,3.0146,3.0276,278286.0,1.654767e+12,8.445901e+05,2822.0,139859.0,4.245051e+05,3.021750,0.0439,2.9891,6.0,2.9513,4.0,1.0,2.0
11909,1.654767e+12,3.0290,3.0549,3.0278,3.0522,221283.0,1.654767e+12,6.730013e+05,2202.0,127075.0,3.865470e+05,3.034800,0.0271,2.9891,7.0,2.9513,5.0,1.0,2.0


In [ ]:
# Columns, note that all of the data are at open time of klines before buy
# time, price, MA(4, 16, 64, 256), TR, ATR(Average True Range, 4, 16, 64, 256), volume, average volume, 
# last top, steps after top, last bottom, Earn